В работе описана модель Transformer-XL ("extra long") - модификация архитектуры и способа обучения [трансформера]($Attention Is All You Need$), которая позволяет работать с более длинными текстами (или другими последовательностями).

### Long-range dependencies

В задачах NLP под понятием [long-range dependencies](https://en.wikipedia.org/wiki/Long-range_dependence) (дальние зависимости) понимаются смысловые связи между участками текста, далеко отстоящими друг от друга. Иногда важно наделить модель способностью улавливать такие связи. Например, в задаче суммаризации модель обучается составлять краткий пересказ текста. Если дойдя до 1000-го слова модель "забудет" то, что было в начале, то ей вероятно не удастся составить качественный пересказ.

Рекуррентные сети (такие как LSTM) плохо справляются с такой задачей. Например, в [данной работе]($Sharp Nearby, Fuzzy Far Away: How Neural Language Models Use Context$) сети LSTM подается на вход длинный текст и изучается, как изменение, перестановка и удаление слов в начале этого текста влияет на точность ответа модели (точнее, на [перплексию](https://towardsdatascience.com/perplexity-in-language-models-87a196019a94), то есть экспоненту от кроссэнтропии на тестовом датасете). Выясняется, что в LSTM на ответ влияют только последние ~200 слов в тексте, при этом порядок слов влияет на ответ только в последних ~50 словах. Судя по всему, данная проблема обусловлена большой глубиной LSTM-сетей, то есть длиной пути в графе вычислений от входных к выходным данным. Эта же проблема приводит к [затухающим и взрывающимся градиентам]($Gradient Flow in Recurrent Nets: the Difficulty of Learning Long-Term Dependencies$) при обучении.

В механизме self-attention в трансформере токены обмениваются информацией независимо от расстояния между ними, что может облегчить обучение дальним зависимостям. Однако в процессе обучения языковых моделей-трансформеров (см. [GPT и BERT]($GPT и BERT$)) длина входной последовательности обычно фиксирована, и увеличение ее ведет к квадратичному росту объема вычислений и потребляемой памяти.

Обучив языковую модель-трансформер на фрагментах текстов длиной, например, 512, можно делать инференс на текстах произвольной длины. Однако позиционные эмбеддинги для элементов начиная с 512-го никогда не встречались при обучении, и модель может быть к ним не приспособлена (об этом см. также в [этой статье]($Train Short, Test Long: Attention with Linear Biases Enables Input Length Extrapolation$)).

### Transformer-XL

Пусть мы обучаем трансформер на фрагментах длиной $N$ (обычно $N=512$), взятых их текстов существенно большей длины. Если текст имеет длину $10N$, то его можно разбить на $10$ фрагментов. Подход авторов данной статьи заключается в том, что при обработке $t$-го фрагмента $n$-й слой self-attention принимает на вектора $(n-1)$-го слоя как из $t$-го фрагмента, так и из $(t-1)$-го фрагмента.

Рассмотрим этот механизм более подробно. Слой трансформера принимает и возвращает последовательность векторов равной длины. Обозначим за $h_t^n$ массив из $N$ векторов, которые подаются на вход $n$-ному слою при обработке $t$-го фрагмента. В обычном трансформере каждый фрагмент обрабатывается полностью независимо, и слой работает таким образом:

$h_t^{n+1} = \text{TransformerLayer}(h_t^n)$

В Transformer-XL слой трансформера обрабатывает вектора, взятые из предыдущего слоя текущего и предыдущего фрагментов:

$h_t^{n+1} = \text{TransformerLayer}(\text{Concat}(\text{StopGradient}(h_{t-1}^n), h_t^n))[-N\text{:}]$

Здесь операция $\text{Concat}$ склеивает вектора из текущего и предыдущего фрагментов, получая $2N$ векторов. Далее слой $\text{TransformerLayer}$ принимает и возвращает $2N$ векторов. Операция $[-N\text{:}]$, записанная python-нотацией, означает взятие последних $N$ векторов. При этом градиенты между фрагментами не распространяются, что обозначено операцией $\text{StopGradient}$.

<img src="assets/transformerxl.jpg" width="700" align="center">

Нераспространение градиентов между фрагментами означает, что можно сначала делать прямой и обратный проход по первому фрагменту, затем по второму и так далее. В операции self-attention в слое трансформера каждый вектор играет роль запроса, ключа и значения (key, query, value). Выходные вектора, соответствующие входным векторам предыдущего фрагмента, отбрасываются, поэтому можно считать, что запросами являются только вектора, пришедшие из текущего фрагмента, а ключами и значениями - вектора из обоих фрагментов. Это означает, что вектор-эмбеддинг каждого токена текущего фрагмента обновляется под действием векторов токенов текущего и предыдущего фрагментов.

Авторы называют архитектуру рекурентной с той оговоркой, что в данном случае рекуррентная зависимость имеется между текущим слоем текущего фрагмента и предыдущим слоем предыдущего фрагмента. Имея $L$ слоев, модель может заглянуть на $L$ фрагментов назад для получения ответа на текущем фрагменте. Авторы проводят аналогию между своим подходом и методом [truncated backpropagation through time]($Recurrent neural network based language model$) (2010).

Выходные векторы, обозначенные на изображении серым, не используются, поэтому их рассчитывать не нужно. В качестве queries нужно брать векторы текущего фрагмента, а в качестве keys и values - векторы предыдущего фрагмента. В общее общем случае, можно использовать не один предыдущий фрагмент, а много фрагментов, насколько позволяет объем памяти (также во время инференса можно использовать больше фрагментов, чем во время обучения). Объем требуемых вычислений и памяти зависит линейно от размера памяти.

В целом данная структура связей между слоями и фрагментами напоминает структуру связей в трансформере, если маска self-attention является [нижней двудиагональной](https://en.wikipedia.org/wiki/Bidiagonal_matrix) матрицей. Поэтому архитектуру Transformer-XL можно описать как "трансформер в трансформере".

<img src="assets/transformerxl2.jpg" width="450" align="center">

### Относительные позиционные эмбеддинги

Слой Transformer-XL обрабатывает вектора из текущего и предыдущего фрагментов. Если позиционный эмбеддинг будет определяться индексом токена внутри фрагмента, то $i$-й токен текущего фрагмента будет иметь тот же позиционный эмбеддинг, что и $i$-й токен предыдущего фрагмента. Тогда нельзя будет различить, какие токены относятся к текущему фрагменту, а какие к предыдущему.

Для решения проблемы авторы предлагают заменить абсолютные позиционные эмбеддинги относительными.

Сначала рассмотрим стандартный трансформер. $i$-й входной вектор первого слоя трансформера можно обозначить как сумму контентного эмбеддинга и позиционного эмбеддинга: $E_i + U_i$. Авторы однако используют это же выражение для $i$-го входного вектора *каждого* слоя. 
Это можно обосновать тем, что каждый слой трансформера работает аддитивно, и можно считать, что он прибавляет слагаемые к $E_i$, не трогая $U_i$. Механизм self-attention рассчитывает скалярные произведения (attention logits) следующим образом:

$\text{AttentionLogit}_{i,j} = (E_i + U_i)^T W_q^T W_k (E_j + U_j)$

Здесь матрицы $W_q$ и $W_k$ являются линейными преобразованиями, выполняемыми над запросами и ключами. Скалярное произведение, как и любое матричное умножение, линейно, поэтому данное выражение равно сумме 4 слагаемых:

$\text{AttentionLogit}_{i,j} = \underbrace{E_i^T W_q^T W_k E_j}_{(a)} + \underbrace{E_i^T W_q^T W_k U_j}_{(b)} + \underbrace{U_i^T W_q^T W_k E_j}_{(c)} + \underbrace{U_i^T W_q^T W_k U_j}_{(d)}$

Авторы предлагают использовать позиционные эмбеддинги не для кодирования абсолютных позиций, а для кодирования смещений между двумя позициями. Абсолютный позиционный эмбеддинг $U_j$ заменяется относительным позиционным эмбеддингом $R_{i-j}$, а абсолютный позиционный эмбеддинг $U_i$ заменяется относительным позиционным эмбеддингом $R_0$ (поскольку смещение между $i$-м и $i$-м элементом равно $0$).

$\underbrace{E_i^T W_q^T W_k E_j}_{(a)} + \underbrace{E_i^T W_q^T W_k R_{i-j}}_{(b)} + \underbrace{R_0^T W_q^T W_k E_j}_{(c)} + \underbrace{R_0^T W_q^T W_k R_{i-j}}_{(d)}$

Далее, поскольку выражение $R_0^T W_q^T$ не зависит от $i$, его можно заменить обучаемым вектором. При этом авторы используют разные вектора для слагаемых $(c)$ и $(d)$ в формуле. И наконец, авторы предлагают заменить матрицу $W_k$ на две разные: одну использовать в слагаемых $(a)$ и $(c)$, другую в слагаемых $(b)$ и $(d)$, поскольку в слагаемых $(a)$ и $(c)$ происходит работа с контентными эмбеддингами, а в слагаемых $(b)$ и $(d)$ с позиционными эмбеддингами, а это разные по смыслу данные. В итоге мы приходим к следующей формуле:

$\text{RelativeAttentionLogit}_{i,j} = \underbrace{E_i^T W_q^T W_{k,E} E_j}_{(a)} + \underbrace{E_i^T W_q^T W_{k,R} R_{i-j}}_{(b)} + \underbrace{u^T W_{k,E} E_j}_{(c)} + \underbrace{v^T W_{k,R} R_{i-j}}_{(d)}$

В слагаемом $(a)$ сопоставляются контентные эмбеддинги, в слагаемом $(b)$ сопоставляется текущий ($i$-й) контентный эмбеддинг и эмбеддинг смещения, слагаемое $(c)$ определяется только текущим контентным эмбеддингом, слагаемое $(d)$ определяется только смещением.

> Each term has an intuitive meaning: term $(a)$ represents content-based addressing, term $(b)$ captures a content-dependent positional bias, term $(c)$ governs a global content bias, and $(d)$ encodes a global positional bias.

В более ранней работе ([Shaw et al., 2018]($Self-Attention with Relative Position Representations$)), где также предлагались относительные позиционные эмбеддинги, использовались только слагаемые $(a)$ и $(b)$, а выражение $W_{k,R} R_{i-j}$ объединялось в единую обучаемую матрицу. Авторы текущей работы выступают против объединения $W_{k,R} R_{i-j}$ в обучаемую матрицу, поскольку такой подход не позволяет использовать синусоидальные эмбеддинги, которые предположительно должны иметь свойство обобщаться на длины большие тех, что встречались при обучении, что важно в Transformer-XL.

Относительные позиционные эмбеддинги встраиваются в Transformer-XL естественным образом: ими можно кодировать как смещение между токенами одного фрагмента, так и смещение между токенами разных фрагментов.

### Результаты

Для достижения SOTA-результатов на ряде задач авторы дополняют Transformer-XL еще несколькими деталями: [adaptive input representations]($Adaptive Input Representations for Neural Language Modeling$), [variational dropout]($A Theoretically Grounded Application of Dropout in Recurrent Neural Networks$) и [weight average]($Regularizing and Optimizing LSTM Language Models$).

Используя достаточно непростой метод подсчета, названный RECL (Relative Effective Context Length, см. Appendix C), авторы показывают, что модель Transformer-XL способна эффективно использовать более длинный контекст, чем аналоги. Приведенный ниже список отсортирован по уменьшению эффективной длины контекста:

- Transformer-XL
- Transformer-XL с другим вариантном относительного позиционного кодирования, описанным в [Shaw et al., 2018]($Self-Attention with Relative Position Representations$)
- Трансформер без рекуррентности, но с предложенным в данной статье вариантом относительного позиционного кодирования
- Стандартный трансформер

Нужно отметить, что в любом трансформере (даже без рекуррентности) есть техническая возможность учитывать контекст любой длины при инференсе, что и используется в данном сравнении.

Также показано, что Transformer-XL имеет большую эффективную длину контекста, чем LSTM. Приведенный ниже список также отсортирован по уменьшению эффективной длины контекста:

- Transformer-XL
- [QRNN]($An Analysis of Neural Language Modeling at Multiple Scales$)
- LSTM

Объединение обоих списков невозможно из-за особенностей метода сравнения (см. Appendix C).

Получается, что оба предложенных авторами метода (относительное позиционное кодирование, рекуррентность) повышают эффективность использования длинного контекста, то есть способности выдавать предсказание с учетом слов, далеко отстоящих от текущего.

### Визуализация внимания

Авторы анализируют Transformer-XL, состоящий из 16 слоев по 10 голов, с памятью длиной 640 (то есть каждый слой принимает на вход вектора из текущего фрагмента и 640 векторов из предыдущих фрагментов).

На изображении ниже для каждой из 160 голов (вертикальная ось) считается среднее значение внимания, которое направлено на 640 предыдущих токенов. Как можно видеть, большинство голов обращают внимание лишь на ближний контекст, но некоторые обращают внимание и на дальний контекст.

<img src="assets/transformerxl3.jpg" width="1000" align="center">

Затем выбирается три головы, которые в наибольшей степени учитывают дальний контекст, а именно 8-я голова 1-го слоя, 7-я голова 8-го слоя и 8-я голова 16-го слоя, и анализируются по отдельности: выбирается текст из датасета WikiText-103 и для него строится матрица внимания, рассчитанная каждой головой. На изображениях ниже по вертикальной оси - индекс токена, откуда направлено внимание, по горизонтальной оси - индекс токена, куда направлено внимание.

<img src="assets/transformerxl4.jpg" width="1000" align="center">

Далее комментарии авторов:

> For the head 8 in the 1st layer, we see an almost uniform attention over the entire memory span. This is quite intuitive, as lower-level layers needs to screen the entire memory span to decide where to focus for higher-level layers.

> For the head 78 in the 8th layer (a middle-level layer), we see a very sparse attention pattern scattered in all ranges of the memory. Again, this well fits our intuition that as information accumulates, the network may focus on some particular position with special interests.

> For the head 158 in the 16th layer (i.e. the last layer), each target location (corresponding to each row) has its own distinct sparse focus, differing from head 78 where target locations largely share the same attentive location in memory. Meanwhile, the pattern is also different from the case of head 8, where a few locations are clearly attended more than others.